In [101]:
# import libraries and env variables
import requests
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()
APP_TOKEN = os.environ.get("X-App-Token")


In [140]:
# initialize parameters
start_date = '2023-11-06T00:00:00.000'
end_date = '2023-11-19T23:59:59.999'
soql_date = f"where=date_of_occurrence between '{start_date}' and '{end_date}'"
limit = 1000

response_data = []

for i in range(3):
    offset = i * limit # offset = 0, 1000, 2000, etc.
    response = requests.get(f"https://data.cityofchicago.org/resource/x2n5-8w5q.json?"
                            f"$$app_token={APP_TOKEN}"
                            f"&$order=:id"  
                            f"&${soql_date}"
                            f"&$limit={limit}"
                            f"&$offset={offset}"
                            f"&$select=:*,*")

    if not response.status_code==200:
        raise Exception

    response_data.extend(response.json())

crime_df = pd.json_normalize(data=response_data)

In [141]:
response.headers

{'Server': 'nginx', 'Date': 'Sun, 14 Jan 2024 14:24:05 GMT', 'Content-Type': 'application/json;charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Access-Control-Allow-Origin': '*', 'ETag': '"YWxwaGEuMTAzMTZfOF8xOTcwMkpBU3lxNVQwZF85S0ZySzI2dHZpNEpRMTBFSQ---gzikTEVSRlK7yRsC6ia4tGGBVPYYI0--gzip--gzip"', 'X-SODA2-Fields': '[":id",":created_at",":updated_at",":version",":@computed_region_awaf_s7ux",":@computed_region_6mkv_f3dw",":@computed_region_vrxf_vc4k",":@computed_region_bdys_3d7i",":@computed_region_43wa_7qmu",":@computed_region_rpca_8um6","case_","date_of_occurrence","block","_iucr","_primary_decsription","_secondary_description","_location_description","arrest","domestic","beat","ward","fbi_cd","x_coordinate","y_coordinate","latitude","longitude","location"]', 'X-SODA2-Types': '["row_identifier","fixed_timestamp","fixed_timestamp","row_version","number","number","number","number","number","number","text","floating_timestamp","text","text","text","text","tex

In [142]:
print(f"Distinct entries: {len(crime_df.groupby(['case_']))}")

Distinct entries: 2998


In [147]:
crime_df = crime_df.drop(columns=[':@computed_region_awaf_s7ux', ':@computed_region_6mkv_f3dw',
       ':@computed_region_vrxf_vc4k', ':@computed_region_bdys_3d7i',
       ':@computed_region_43wa_7qmu', ':@computed_region_rpca_8um6'])

In [149]:
crime_df.columns

Index([':id', ':created_at', ':updated_at', ':version', 'case_',
       'date_of_occurrence', 'block', '_iucr', '_primary_decsription',
       '_secondary_description', '_location_description', 'arrest', 'domestic',
       'beat', 'ward', 'fbi_cd', 'x_coordinate', 'y_coordinate', 'latitude',
       'longitude', 'location.latitude', 'location.longitude',
       'location.human_address'],
      dtype='object')

In [148]:
crime_df

,:id,:created_at,:updated_at,:version,case_,date_of_occurrence,block,_iucr,_primary_decsription,_secondary_description,...,beat,ward,fbi_cd,x_coordinate,y_coordinate,latitude,longitude,location.latitude,location.longitude,location.human_address
0,row-n56f.jhj4-rhq4,2023-11-14T11:02:01.256Z,2023-11-14T11:02:11.508Z,rv-fhmb.32vx_5wns,JG495930,2023-11-06T00:00:00.000,023XX W LELAND AVE,0820,THEFT,$500 AND UNDER,...,1911,47,06,1159788,1931121,41.96671217,-87.687866208,41.96671217,-87.687866208,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
1,row-p4xb~y53j-fuek,2023-11-14T11:02:01.256Z,2023-11-14T11:02:11.508Z,rv-5cs2.q96g.nma6,JG494364,2023-11-06T00:00:00.000,105XX S WALDEN PKWY,0560,ASSAULT,SIMPLE,...,2212,19,08A,1165481,1834708,41.702025415,-87.669672767,41.702025415,-87.669672767,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
2,row-bevv.9vv5~9xk5,2023-11-14T11:02:01.256Z,2023-11-14T11:02:11.508Z,rv-qf9x~yuq5.p9r6,JG495115,2023-11-06T00:00:00.000,074XX W BELMONT AVE,0930,MOTOR VEHICLE THEFT,THEFT / RECOVERY - AUTOMOBILE,...,1631,29,07,1125766,1920416,41.937971562,-87.813201099,41.937971562,-87.813201099,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
3,row-u3gy~wdhi.hd3k,2023-11-14T11:02:01.256Z,2023-11-14T11:02:11.508Z,rv-jk7w~dau9-eci4,JG494379,2023-11-06T00:00:00.000,069XX S HARPER AVE,2820,OTHER OFFENSE,TELEPHONE THREAT,...,332,5,08A,1187589,1859066,41.768370253,-87.587947949,41.768370253,-87.587947949,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
4,row-m3c2~ztbv~kdrh,2023-11-14T11:02:01.256Z,2023-11-14T11:02:11.508Z,rv-ma52_veqi_ib8b,JG493406,2023-11-06T00:00:00.000,098XX S HOXIE AVE,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,...,431,7,07,1195083,1840238,41.71652318,-87.561098665,41.71652318,-87.561098665,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,row-4arz-itc2~eeac,2023-11-19T11:02:13.735Z,2023-11-19T11:02:20.179Z,rv-7xqj.cnht_n74i,JG499108,2023-11-10T14:24:00.000,079XX S GREENWOOD AVE,3730,INTERFERENCE WITH PUBLIC OFFICER,OBSTRUCTING JUSTICE,...,624,8,24,1184929,1852601,41.750692511,-87.597900594,41.750692511,-87.597900594,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
2996,row-a7rf~tykm.f6n4,2023-11-19T11:02:13.735Z,2023-11-19T11:02:20.179Z,rv-2jb3_unrd.mzsx,JG499503,2023-11-10T20:16:00.000,075XX S SOUTH SHORE DR,0460,BATTERY,SIMPLE,...,421,7,08B,1196126,1856148,41.76015578,-87.556752911,41.76015578,-87.556752911,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
2997,row-fpqv_ex4t-ee9t,2023-11-19T11:02:13.735Z,2023-11-19T11:02:20.179Z,rv-tcm8.csck~3v26,JG498762,2023-11-10T06:20:00.000,034XX W 71ST PL,0486,BATTERY,DOMESTIC BATTERY SIMPLE,...,831,17,08B,1154617,1856985,41.763380216,-87.708862109,41.763380216,-87.708862109,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."
2998,row-9g4h~dk7q-vga7,2023-11-19T11:02:13.735Z,2023-11-19T11:02:20.179Z,rv-b7rk~u64w-bm3r,JG500236,2023-11-10T13:30:00.000,008XX S INDEPENDENCE BLVD,0313,ROBBERY,ARMED - OTHER DANGEROUS WEAPON,...,1133,24,03,1151228,1896017,41.870556289,-87.720262929,41.870556289,-87.720262929,"{""address"": """", ""city"": """", ""state"": """", ""zip""..."


In [105]:
# create a dataframe for specific dates with holiday column 

begin_date = "2023-01-01"
end_date = "2024-12-31" 
# ^ parameters to be set by yaml config later

date_df = pd.DataFrame({'Date':pd.date_range(start=begin_date, end=end_date)})

date_df['Day'] = date_df['Date'].dt.day
date_df['Month'] = date_df['Date'].dt.month
date_df['MonthName'] = date_df['Date'].dt.month_name()
date_df['Year'] = date_df['Date'].dt.year
date_df['DayOfWeek'] = date_df['Date'].dt.dayofweek
date_df['DayOfWeekName'] = date_df['Date'].dt.day_name()

In [106]:
holidays_df = pd.concat(map(pd.read_csv, ['raw_data/holidays/2023.csv', 'raw_data/holidays/2024.csv']))
# ^ change code above to run as for loop for each file inside the holidays folder

holidays_df = holidays_df.rename(columns={'Name': 'HolidayName'})

In [107]:
holidays_df['Date'] = pd.to_datetime(holidays_df['Date'])

In [108]:
date_merge_df = pd.merge(left=date_df, right=holidays_df, on=["Date"], how="left")

In [133]:
date_merge_df

,Date,Day,Month,MonthName,Year,DayOfWeek,DayOfWeekName,HolidayName
0,2023-01-01,1,1,January,2023,6,Sunday,NaN
1,2023-01-02,2,1,January,2023,0,Monday,New Year's Day
2,2023-01-03,3,1,January,2023,1,Tuesday,NaN
3,2023-01-04,4,1,January,2023,2,Wednesday,NaN
4,2023-01-05,5,1,January,2023,3,Thursday,NaN
...,...,...,...,...,...,...,...,...
726,2024-12-27,27,12,December,2024,4,Friday,NaN
727,2024-12-28,28,12,December,2024,5,Saturday,NaN
728,2024-12-29,29,12,December,2024,6,Sunday,NaN
729,2024-12-30,30,12,December,2024,0,Monday,NaN
